<a href="https://colab.research.google.com/github/wikiban43/Misinformation-Analysis/blob/main/tweet_rt_flow_analysis_icwsm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get the tweets and RTs over time for an event

In [1]:
import json
import requests
import os
import json
import time
import pandas as pd
from datetime import datetime,timedelta
import time
import pickle
from google.colab import drive
drive.mount('/content/drive')



#bearer_token = 'AAAAAAAAAAAAAAAAAAAAAIJxNAEAAAAAHZ9CAu5ihxMmJ0LvjF75lQ3E7%2Fo%3DBLVYPaKVU1iNasbucVGf9RY5WTNESDiRFjcuu3cSRjuWq82Zs3'
#bearer_token = "AAAAAAAAAAAAAAAAAAAAAD%2BcYgEAAAAAj5%2Bw705OKXy%2F%2FVYhB9j6jjwVIik%3DKEdmgWebZLI2zXFkOuYFRYzLbsyOrkjMNhqHWwP9f3GQvGoRbD"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAD%2BcYgEAAAAArlwQRh89reyAIz1iwuq3PiuDieM%3DRQOOeEaVxUTwgdOZiVnNiPbLpD48ct2g2Vp9BskfI6Hsw3gKyx"
endpoint = "https://api.twitter.com/2/tweets/search/all"
headers = {"Authorization": "Bearer {}".format(bearer_token)}

Mounted at /content/drive


In [42]:
#Get CAA-NRC and Budget-2022 tweets
path = 'drive/My Drive/MSR/icwsm_datasets/'
#df_caa = pd.read_csv(path+"caa_nrc.csv")
#df_caa = pd.read_csv(path+"budget2022.csv")
df_caa = pd.read_csv(path+"ipl2022final.csv")
#df_caa.head()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning:

Columns (0,3,5,8,10,12,15,19,38,41,42,43) have mixed types.Specify dtype option on import or set low_memory=False.



In [43]:
df_caa.head()

,Unnamed: 0,text,conversation_id,author_id,created_at,id,entities.annotations,entities.urls,public_metrics.retweet_count,public_metrics.reply_count,...,geo.name,geo.country_code,author.entities.description.mentions,author.entities.description.hashtags,entities.cashtags,author.entities.description.cashtags,attachments.poll_ids,withheld.copyright,withheld.country_codes,author.withheld.country_codes
0,0,Light And Sound Show Enthralls Capacity Crowd ...,1.531425e+18,1413183985207975943,2022-05-30,1531425074460385283,"[{'start': 82, 'end': 90, 'probability': 0.648...","[{'start': 92, 'end': 115, 'url': 'https://t.c...",0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,@Pogba11715375 @klrahul @gujarat_titans @rajas...,1.531149e+18,1422265507571331072,2022-05-30,1531424926523342848,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,#MonacoGP #ฝีดาษลิง #stadedefrance #IPLFinal #...,1.531425e+18,347225664,2022-05-30,1531424900786884614,NaN,"[{'start': 130, 'end': 153, 'url': 'https://t....",0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Money rained on Jos Buttler, he earned so much...",1.531424e+18,1503462245568692225,2022-05-30,1531424484573515779,"[{'start': 16, 'end': 26, 'probability': 0.832...","[{'start': 110, 'end': 133, 'url': 'https://t....",0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"@imVkohli Beta #Chokli , jitna showoff aur ove...",1.530542e+18,1246178680507740160,2022-05-30,1531424323575197696,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df_caa = df_caa[['id','created_at','text','referenced_tweets','public_metrics.retweet_count']]

KeyError: ignored

In [6]:
#Get the date wise plots for the overall number of tweets 
from datetime import datetime
def get_date(string):
  date = datetime.strptime(string, '%Y-%m-%dT%H:%M:%S.%fZ')
  #date = str(date.month)+"/"+str(date.year)
  return date.date()


df_caa["created_at"] = df_caa["created_at"].apply(lambda x: get_date(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [25]:
df_caa.drop_duplicates(inplace=True)
df_caa.reset_index(inplace=True)
df_caa_g = df_caa.groupby(["created_at"])["text"].count()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [26]:
df_caa_g = df_caa_g.reset_index()

In [17]:
df_caa["referenced_tweets"].shape

(104484,)

In [ ]:
def isNaN(string):
  return string != string
import numpy as np



df_caa["referenced_tweets"] = df_caa["referenced_tweets"].fillna("None")
orig2latestrttime=dict()
for i in range(0,df_caa.shape[0]):
  #print(i)
  #print(df_main2["referenced_tweets"][i])
  if df_caa["referenced_tweets"][i]=="None":
    orig2latestrttime[str(df_caa["id"][i])] = (df_caa["created_at"][i],df_caa["public_metrics.retweet_count"][i])
  else:
    #print(df_main2["referenced_tweets"][i])
    #try:
    orig_id = df_caa["referenced_tweets"][i].split(":")[2].replace('}]','')
    orig_id = orig_id.replace('\'','')
    orig_id = orig_id.split(',')[0]
    #print(orig_id)
    #If the original ID is already there, replace the retweet count iff it's greater than the old value. If it's not there, put the new retweet_count.
    if orig_id in orig2latestrttime.keys():
      if int(orig2latestrttime[orig_id][1]) <= int(df_caa["public_metrics.retweet_count"][i]):
        #print("Replacing:"+str(orig2latestrttime[orig_id][1])+" with "+str(df_main2["retweet_count"][i]))
        #print("Replacing:"+str(orig2latestrttime[orig_id][0])+" with "+str(df_main2["created_at"][i]))
        orig2latestrttime[orig_id] = (df_caa["created_at"][i],df_caa["public_metrics.retweet_count"][i])
    else:
      orig2latestrttime[orig_id] = (df_caa["created_at"][i],df_caa["public_metrics.retweet_count"][i])
    
    #except:
    #  print(i)
    #  continue
print("COMPLETE")

In [35]:
pickle.dump(orig2latestrttime,open(path+"orig2latestrttime_caa.pickle","wb"))

OSError: ignored

In [ ]:
orig2latestrttime = pickle.load(open(path+"orig2latestrttime.pickle","rb"))

In [36]:
id_l=[]
date_l=[]
rt_count_l=[]

for key in orig2latestrttime:
  id_l.append(key)
  date_l.append(orig2latestrttime[key][0])
  rt_count_l.append(orig2latestrttime[key][1])

df_rt_date_count = pd.DataFrame()
df_rt_date_count["id"] = id_l
df_rt_date_count["date"] = date_l
df_rt_date_count["retweet count"] = rt_count_l


In [37]:
df_rt_date_count_g = df_rt_date_count.groupby(["date"])["retweet count"].sum().reset_index()

In [38]:
#CORRECT PLOT OF THE NUMBER OF TWEETS AND RETWEETS RECEIVED OVER TIME
import plotly.graph_objects as go
from plotly.subplots import make_subplots

trace1 = go.Line(
    x=df_rt_date_count_g["date"],
    y=df_rt_date_count_g["retweet count"],
    name="Total number of retweets",
    marker=dict(
        color='rgb(34,163,192)'
               )
)
trace2 = go.Line(
    x=df_caa_g["created_at"],
    y=df_caa_g["text"],
    name='Number of tweets',
    yaxis='y2'

)

fig = make_subplots(specs=[[{"secondary_y": True}]])
#fig.add_trace(trace1)
#fig.add_trace(trace2,secondary_y=False)
fig.add_trace(trace1,secondary_y=False)
fig.add_trace(trace2,secondary_y=True)
fig['layout'].update(height = 600, width = 1000, xaxis=dict(
      tickangle=-90
    ))

fig.update_yaxes(title_text="<b>Number of retweets</b> ", secondary_y=False)
fig.update_yaxes(title_text="<b>Number of tweets</b> ", secondary_y=True)

fig.update_layout(
xaxis = dict(
tickfont = dict(size=20),titlefont = dict(size=25)),
yaxis = dict(
tickfont = dict(size=20),titlefont = dict(size=25))
)
fig.update_layout(legend=dict(yanchor="top", y=0.9, xanchor="left", x=0))
#iplot(fig)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


